In [38]:
import pandas as pd
from tqdm.auto import tqdm
from transformers import AutoModelForCausalLM

In [3]:
mistral = AutoModelForCausalLM.from_pretrained("mistralai/Mistral-7B-v0.1")

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

let's check a model diff

In [4]:
model_id = "HuggingFaceH4/zephyr-7b-beta"

In [5]:
model = AutoModelForCausalLM.from_pretrained(model_id)

Loading checkpoint shards:   0%|          | 0/8 [00:00<?, ?it/s]

In [6]:
model

MistralForCausalLM(
  (model): MistralModel(
    (embed_tokens): Embedding(32000, 4096, padding_idx=2)
    (layers): ModuleList(
      (0-31): 32 x MistralDecoderLayer(
        (self_attn): MistralAttention(
          (q_proj): Linear(in_features=4096, out_features=4096, bias=False)
          (k_proj): Linear(in_features=4096, out_features=1024, bias=False)
          (v_proj): Linear(in_features=4096, out_features=1024, bias=False)
          (o_proj): Linear(in_features=4096, out_features=4096, bias=False)
          (rotary_emb): MistralRotaryEmbedding()
        )
        (mlp): MistralMLP(
          (gate_proj): Linear(in_features=4096, out_features=14336, bias=False)
          (up_proj): Linear(in_features=4096, out_features=14336, bias=False)
          (down_proj): Linear(in_features=14336, out_features=4096, bias=False)
          (act_fn): SiLU()
        )
        (input_layernorm): MistralRMSNorm()
        (post_attention_layernorm): MistralRMSNorm()
      )
    )
    (norm): Mist

In [92]:
def check_tensor_dims(t1, t2):
    dim_where_not_equal = []
    for s1, s2 in zip(t1.shape, t2.shape):
        dim_where_not_equal.append(s1 != s2)
    return dim_where_not_equal

In [104]:
def _compute_diff(model, ref_model):
    total_params = sum([p.numel() for p in model.parameters()])
    total_layers = sum([1 for p in model.parameters()])
    diffs = {}
    pbar = tqdm(zip(model.state_dict().items(), ref_model.state_dict().items()), total=total_layers)
    for (k_base, p_base), (k_ft, p_ft) in pbar:
        assert k_base == k_ft,"Models don't match, compare to the same model" 
        dim_where_not_equal = check_tensor_dims(p_base, p_ft)
        if sum(dim_where_not_equal):
            print(f"Warning: {k_base} has shape: {p_base.shape} != {p_ft.shape}")
            # we only allow diverging dims on the first and last layers (embedding and head)
            if dim_where_not_equal[0]:
                min_dim = min(p_base.shape[0], p_ft.shape[0])
                p_base = p_base[:min_dim]
                p_ft = p_ft[:min_dim]
        
        _diff = (p_base - p_ft).abs()
        diffs[k_base] = {
            "abs":    _diff.sum().item(), 
            "mean":   _diff.mean().item(),
            # "median": _diff.median().item()
        }
    return diffs

In [94]:
diffs = _compute_diff(model, mistral)

  0%|          | 0/291 [00:00<?, ?it/s]

In [105]:
def aggregate_layer_results(data, levels=3):
    # Initialize a dictionary to hold the aggregated results
    aggregated_results = {}

    # Iterate through each key-value pair in the input data
    for key, value in data.items():
        # Identify and extract the layer name
        layer_name_parts = key.split('.')
        layer_name = ".".join(layer_name_parts[:levels])
        
        # Initialize the layer in the aggregated results if not present
        if layer_name not in aggregated_results:
            aggregated_results[layer_name] = {'abs': 0, 'mean': 0, 'count': 0}

        # Aggregate the results
        aggregated_results[layer_name]['abs'] += value['abs']
        aggregated_results[layer_name]['mean'] += value['mean']
        aggregated_results[layer_name]['count'] += 1

    # Calculate the average of the means for each layer
    for layer, stats in aggregated_results.items():
        stats['mean'] /= stats['count']
        del stats['count']  # Remove count as it's no longer needed

    return aggregated_results

In [96]:
diffs = _compute_diff(model, mistral)
layer_results = aggregate_layer_results(diffs, levels=3)

  0%|          | 0/291 [00:00<?, ?it/s]

In [106]:
def to_df(data):
    df = pd.DataFrame.from_dict(data, orient='index')
    def apply_color_scale(df):
        return df.style.background_gradient(cmap='Reds', subset=["abs"],  axis=0)\
                       .background_gradient(cmap='Reds', subset=["mean"], axis=0)
    # Apply the color scale to the DataFrame
    styled_df = apply_color_scale(df)
    return styled_df

In [98]:
df = to_df(layer_results)
df

,abs,mean
model.embed_tokens.weight,18778.666016,0.000143
model.layers.0,22515.555466,0.000098
model.layers.1,24025.562738,0.000101
model.layers.2,21089.673368,0.000074
model.layers.3,21632.520883,0.000078
model.layers.4,20266.105537,0.000071
model.layers.5,20565.547354,0.000073
model.layers.6,19795.855812,0.000069
model.layers.7,20123.770569,0.000070
model.layers.8,19646.641113,0.000067


In [107]:
def compute_diff(model_id, ref_model_id, levels=3):
    model = AutoModelForCausalLM.from_pretrained(model_id)
    ref_model = AutoModelForCausalLM.from_pretrained(ref_model_id)
    diffs =  _compute_diff(model, ref_model)
    layer_results = aggregate_layer_results(diffs, levels=levels)
    return to_df(layer_results)

In [100]:
res_df = compute_diff("mistralai/Mistral-7B-Instruct-v0.1", "mistralai/Mistral-7B-v0.1")

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

  0%|          | 0/291 [00:00<?, ?it/s]

In [101]:
res_df

,abs,mean
model.embed_tokens.weight,57131.996094,0.000436
model.layers.0,94654.652822,0.000623
model.layers.1,100114.943449,0.000876
model.layers.2,104377.885955,0.000889
model.layers.3,104298.319109,0.000785
model.layers.4,103622.169692,0.000808
model.layers.5,103278.184989,0.000899
model.layers.6,102811.395339,0.000786
model.layers.7,102527.069824,0.000783
model.layers.8,102125.578369,0.000770


In [102]:
tek_model = "teknium/OpenHermes-2.5-Mistral-7B"

res_df = compute_diff("teknium/OpenHermes-2.5-Mistral-7B", "mistralai/Mistral-7B-v0.1")

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

  0%|          | 0/291 [00:00<?, ?it/s]

In [103]:
res_df

,abs,mean
model.embed_tokens.weight,4657.928223,0.000036
model.layers.0,6601.050701,0.000020
model.layers.1,9378.093040,0.000028
model.layers.2,11159.152022,0.000039
model.layers.3,11977.889692,0.000041
model.layers.4,12613.707894,0.000043
model.layers.5,13588.014727,0.000047
model.layers.6,13932.616326,0.000048
model.layers.7,14594.487610,0.000050
model.layers.8,14904.710556,0.000052


In [ ]:
model_id = "Open-Orca/Mistral-7B-OpenOrca"
res_df = compute_diff(model_id, "mistralai/Mistral-7B-v0.1")
res_df

config.json:   0%|          | 0.00/623 [00:00<?, ?B/s]

pytorch_model.bin.index.json:   0%|          | 0.00/23.9k [00:00<?, ?B/s]

pytorch_model-00001-of-00002.bin:   0%|          | 0.00/9.94G [00:00<?, ?B/s]